In [29]:
import sys
sys.path.append('../Data')

import pandas as pd
import numpy as np
import time
from CommentDataset import CommentDataset

In [30]:
comment_list_name = "20230627_mynumbercard_untenmenkyosyou"
comment_list_path = "2023_06_27/" + comment_list_name + ".csv"

In [31]:
# 指定すべき文字コードはファイルによる

df = pd.read_csv("../Data/yahoo/"+comment_list_path, encoding="utf-8", header=None)
comment_list = df[0][1:].to_numpy().tolist()
comment_dataset = CommentDataset(comment_list)
comment_dataset.delete_comment_specified_string(50)
comment_dataset.formatted_input_hlda(appear_tagging_list=["名詞", "形容詞", "動詞"], stop_tagging_list=["助詞", "助動詞", "非自立"])

In [32]:
from hlda.sampler import HierarchicalLDA

#default
n_samples = 500       # no of iterations for the sampler
alpha = 10.0          # smoothing over level distributions
gamma = 1.0           # CRP smoothing parameter; number of imaginary customers at next, as yet unused table
eta = 0.1             # smoothing over topic-word distributions
num_levels = 3        # the number of levels in the tree
display_topics = 50   # the number of iterations between printing a brief summary of the topics so far
n_words = 10           # the number of most probable words to print for each topic after model estimation
with_weights = False  # whether to print the words with the weights

In [33]:
hlda = HierarchicalLDA(comment_dataset.comment_list, comment_dataset.voc, alpha=alpha, gamma=gamma, eta=eta, num_levels=num_levels)
hlda.estimate(n_samples, display_topics=display_topics, n_words=n_words, with_weights=with_weights)

HierarchicalLDA sampling

.................................................. 50
topic=0 level=0 (documents=528): マイナンバーカード, れる, 国民, できる, 思う, 免許証, 運転免許証, 言う, カード, 保険証, 
    topic=1 level=1 (documents=113): れる, 手続き, 使う, 運転免許, 役所, 持つ, 記事, できる, 作る, いう, 
        topic=2 level=2 (documents=38): 紛失, 発行, 身分証, 場合, かかる, 病院, 万が一, 顔写真, そう, 行く, 
        topic=24 level=2 (documents=4): 中国, 移住, 好き, 役所, 一緒に, 免許更新, 二階, 老害, 真似, 様々, 
        topic=61 level=2 (documents=8): 支持率, 政治家, 公的, 有る, 投票, 私たち, 総理, 岸田, 無視, 何処, 
        topic=91 level=2 (documents=11): コード, 基本, 年寄り, 限り, 中国, 整備, なな, 音痴, 正す, 運転経歴証明書, 
        topic=101 level=2 (documents=5): 対応, 家畜, 世界の支配者, 人類, 悪用, 2024年, 違い, 個人番号, 命令, 記載, 
        topic=104 level=2 (documents=14): 作る, 開設, 銀行口座, 契約, 口座, 銀行, 携帯電話, ライセンス, 新しい, 都市銀行, 
        topic=107 level=2 (documents=4): 複雑, 自信, 成り立つ, 廃止, 成す, 立案, パターン, 反対派, 一部の人, 説得, 
        topic=109 level=2 (documents=8): 必要性, 来る, 止める, 無い, 諦める, お巡りさん, ガラケー, 経つ, 見返す, 近い, 
        topic=139 level=2 (documents=4): 友達,

In [36]:
from model.HldaModel import ExpandHldaModel
import os

create_date = "2023_06_27"
parameter_name = "default"
model_name = comment_list_name + "_" + parameter_name + ".pickle"
model_dir = "model/pickle/" + create_date + "/" +model_name
create_date_path = "model/pickle/" + create_date

if not os.path.exists(create_date_path):
    os.makedirs(create_date_path)

expandHlda = ExpandHldaModel(pickle_path=hlda)
expandHlda.save_zipped_pickle(filename=model_dir)